In [1]:
  %%capture
!pip install datasets
!pip install transformers

In [2]:
# cite simcse
# cite sentence-transformers https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

## My Dataset

In [101]:
colab = True

In [102]:
label = "final_pred_modified"

In [103]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
#df = pd.read_excel("data/survey_data/labelled_data/sat_data_combined_vertical_pred_final.xlsx")
if colab:
  df = pd.read_excel("/content/drive/MyDrive/Imperial College London/Individual Project/Jupyter/data_labelled.xlsx")
else:
  df = pd.read_excel("data/survey_data/labelled_data/data_labelled.xlsx")

In [106]:
df[label] = df[label].astype(int)

In [107]:
X = df['response']
y = df[label]

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

In [109]:
texts = list(X_train)
texts_test = list(X_test)
texts_val = list(X_val)

In [110]:
df[label].value_counts()

2    871
0    824
1    673
Name: final_pred_modified, dtype: int64

In [111]:
X_train

526     Are you aware of on-goings within your local c...
92      Congratulations! You have shown foresighted co...
963     I would like to emphasise that the purpose of ...
909     Congratulations! You have shown tender compass...
2242    This is a fantastic start but bear in mind tha...
                              ...                        
1033     As a first step : shall we continue with ESA?...
1731    I would like to emphasize that the purpose of ...
763     Think about how you plan to achieve this. When...
835                        How are you feeling right now?
1653    Let's talk a little bit more about you - Do yo...
Name: response, Length: 1657, dtype: object

## Model prediction

In [112]:
%%capture
#!pip install transformer[sentencepiece]

In [114]:

#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
#tokenizer = AutoTokenizer.from_pretrained("roberta-base")
#model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
#tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
#model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/ma

In [115]:
from torch.utils.data import Dataset, DataLoader
class EmpatheticDataset(Dataset):
    def __init__(self, X, y, with_label=True):
        self.X = X.reset_index()['response']
        self.y = y.reset_index()[label]
        self.text = list(X.values)
        encoded_input = tokenizer(self.text, padding=True, truncation=True, return_tensors="pt")
        encoded_input['text'] = self.text
        encoded_input['label'] = torch.tensor(y.values)
        if with_label:
            try:
                self.new_output = [{'label': label, 'text': text,'input_ids': input_ids, \
                           'token_type_ids': token_type_ids, 'attention_mask': attention_mask} \
                           for label, text, input_ids, token_type_ids, attention_mask in zip(encoded_input['label'], \
                                                                              encoded_input['text'], \
                                                                              encoded_input['input_ids'], \
                                                                              encoded_input['token_type_ids'], \
                                                                              encoded_input['attention_mask'])]
            except:
                self.new_output = [{'label': label, 'text': text,'input_ids': input_ids, \
                           'attention_mask': attention_mask} \
                           for label, text, input_ids, attention_mask in zip(encoded_input['label'], \
                                                                              encoded_input['text'], \
                                                                              encoded_input['input_ids'], \
                                                                              encoded_input['attention_mask'])]
        else:
            try:
                self.new_output = [{'text': text,'input_ids': input_ids, \
                           'token_type_ids': token_type_ids, 'attention_mask': attention_mask} \
                           for text, input_ids, token_type_ids, attention_mask in zip(encoded_input['text'], \
                                                                              encoded_input['input_ids'], \
                                                                              encoded_input['token_type_ids'], \
                                                                              encoded_input['attention_mask'])]
            except:
                self.new_output = [{'text': text,'input_ids': input_ids, \
                           'attention_mask': attention_mask} \
                           for text, input_ids, attention_mask in zip(encoded_input['text'], \
                                                                              encoded_input['input_ids'], \
                                                                              encoded_input['attention_mask'])]
            
        
 
    def __len__(self):
        return len(self.X)
   
    def __getitem__(self,idx):
        try:
            return self.new_output[idx]
        except:
            print(idx)
            print(self.new_output[idx - 1])
            print(self.new_output[idx])
        #return self.X[idx], self.y[idx]
        #features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']

In [116]:
train_data = EmpatheticDataset(X_train, y_train)
test_data = EmpatheticDataset(X_test, y_test)
val_data = EmpatheticDataset(X_val, y_val)

In [117]:
#train_dataloader = DataLoader(train_data, shuffle=True, batch_size=8)
#test_dataloader = DataLoader(test_data, batch_size=8)

In [118]:
from datasets import load_metric

metric = load_metric("accuracy")

In [119]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [120]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [121]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", 
                                  metric_for_best_model='accuracy',
                                  learning_rate=5e-5, #5e-5
                                  per_device_train_batch_size=8, 
                                  per_device_eval_batch_size=8,
                                  adafactor=False,
                                  logging_first_step=True,
                                  logging_steps=100,
                                  num_train_epochs=5,
                                  adam_epsilon=1e-3)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [122]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
    
)

In [123]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1657
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1040
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.936700,0.877542,0.580282,0.580282,0.580282,0.580282
2,0.866600,0.785818,0.647887,0.647887,0.647887,0.647887
3,0.740900,0.755284,0.656338,0.656338,0.656338,0.656338
4,0.708600,0.713397,0.687324,0.687324,0.687324,0.687324
5,0.712100,0.706986,0.690141,0.690141,0.690141,0.690141


***** Running Evaluation *****
  Num examples = 355
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 355
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 355
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertFo

TrainOutput(global_step=1040, training_loss=0.8015521095349238, metrics={'train_runtime': 259.357, 'train_samples_per_second': 31.944, 'train_steps_per_second': 4.01, 'total_flos': 634383408624210.0, 'train_loss': 0.8015521095349238, 'epoch': 5.0})

In [124]:
save_path = "distilbert-base-uncased_full_data.pt"
#torch.save(model.state_dict(), save_path)

In [125]:
# METHOD 1
#save_path = "hf_models/bert-base-uncased_full_data.pt"
#model2 = torch.load(save_path)

In [126]:
# METHOD 2
#trainer.save_model("/content/gdrive/MyDrive/Imperial College London/Individual Project/Model")

In [127]:
# METHOD 3: SAVE in colab
#from google.colab import drive
#drive.mount('/content/gdrive')

## Loading Pre-trained model

In [128]:
# METHOD 1
"""

model2 = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model2.load_state_dict(torch.load(save_path, map_location="cpu"))
trainer2 = Trainer(
    model=model2,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)
"""

'\n\nmodel2 = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)\nmodel2.load_state_dict(torch.load(save_path, map_location="cpu"))\ntrainer2 = Trainer(\n    model=model2,\n    args=training_args,\n    train_dataset=train_data,\n    eval_dataset=val_data,\n    compute_metrics=compute_metrics,\n)\n'

In [129]:
# METHOD 2
#model = AutoModelForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/Imperial College London/Individual Project/Model", local_files_only=True)
#trainer.model = model.cuda()

# Inference

In [130]:
y_pred_hf = trainer.predict(test_data)
y_pred_dl = np.argmax(y_pred_hf[0], axis=1)

***** Running Prediction *****
  Num examples = 356
  Batch size = 8
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


In [131]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_dl))

              precision    recall  f1-score   support

           0       0.82      0.83      0.83       127
           1       0.56      0.30      0.40       105
           2       0.64      0.88      0.74       124

    accuracy                           0.69       356
   macro avg       0.67      0.67      0.65       356
weighted avg       0.68      0.69      0.67       356



## Time spent on inference


In [ ]:
X_time_discard, X_time, y_time_discard, y_time= train_test_split(X, y, test_size=0.01, random_state=0)

In [ ]:
time_data = EmpatheticDataset(X_time, y_time)

In [ ]:
import time

start = time.time()
y_pred_hf = trainer.predict(time_data)
y_pred_dl = np.argmax(y_pred_hf[0], axis=1)
end = time.time()
time_spent = end - start
print(f"Time spent: {time_spent}")
print(f"Inference data size: {len(time_data)}")

***** Running Prediction *****
  Num examples = 24
  Batch size = 8
The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.


Time spent: 0.14568734169006348
Inference data size: 24
